In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
'Deep learning Model creation steps:'
'1. Data processing: '
'   Image Data Generator'
'   Train test split'
'2. Importing the Model'
'3. Customize the Model: '
'4. Create the combination of Input and Output layer'
'5. Compile model'
'6. Train model'
'7. Test Model'

In [14]:
class MyCLRuleMonitor(tf.keras.callbacks.Callback):
  def __init__(self, CL):
    super(MyCLRuleMonitor).__init__()
    self.CL = CL

  def on_epoch_end(self, epoch, logs=None):
    trainScore = logs["accuracy"]
    testScore = logs["val_accuracy"]

    if testScore > trainScore and testScore >= self.CL:
      self.model.stop_training = True

In [2]:
train_image_data = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
test_image_data = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

In [4]:
train_image  = train_image_data.flow_from_directory('cats_and_dogs/cats_and_dogs/train',
                                                    class_mode= 'binary',
                                                    target_size = (128,128)
                                                    )

test_image = test_image_data.flow_from_directory('cats_and_dogs/cats_and_dogs/validation',
                                                 batch_size= 20,
                                                 class_mode = 'binary',
                                                 target_size = (128, 128))

Found 2000 images belonging to 2 classes.
Found 1002 images belonging to 2 classes.


In [7]:
xception = tf.keras.applications.xception.Xception(include_top=False)

In [8]:
xception.summary()

Model: "xception"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, None,      │        864 │ input_layer_1[0]… │
│ (Conv2D)            │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, None,      │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, None,      │          0 │ block1_conv1_bn[… │
│ (Activation)        │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, None,      │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, None,      │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, None,      │          0 │ block1_conv2_bn[… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, None,      │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, None,      │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, None,      │          0 │ block2_sepconv1_… │
│ (Activation)        │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, None,      │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, None,      │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, None,      │      8,192 │ block1_conv2_act… │
│                     │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, None,      │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │        512 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_12 (Add)        │ (None, None,      │          0 │ block2_pool[0][0… │
│                     │ None, 128)        │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, None,      │          0 │ add_12[0][0]    

 Total params: 20,861,480 (79.58 MB)

 Trainable params: 20,806,952 (79.37 MB)

 Non-trainable params: 54,528 (213.00 KB)

In [12]:
# we do not want to touch/train the existing weighs of the model's layer:
for layer in xception.layers:
    layer.trainable  = False

In [10]:
input_layer = tf.keras.layers.Input([128,128,3])

x = xception(input_layer)

x1 = tf.keras.layers.Flatten()(x)
x2 = tf.keras.layers.Dense(activation='relu', units = 128, name = 'h1')(x1)
x3 = tf.keras.layers.Dense(activation='relu', units= 128, name = 'h2')(x2)

output = tf.keras.layers.Dense(activation='sigmoid', units = 1, name = 'output')(x3)

model = tf.keras.models.Model(inputs = input_layer, outputs = output)

In [11]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception (Functional)           │ (None, 4, 4, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ h1 (Dense)                      │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ h2 (Dense)                      │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,072,553 (95.64 MB)

 Trainable params: 25,018,025 (95.44 MB)

 Non-trainable params: 54,528 (213.00 KB)

In [24]:
model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [25]:
model.fit(train_image,
          validation_data = test_image,
          steps_per_epoch = (len(train_image.filenames) // train_image.batch_size),
          validation_steps = (len (test_image.filenames)// test_image.batch_size),
          epochs = 100,
          callbacks = [MyCLRuleMonitor(0.9)]
          )

Epoch 1/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 0.9360 - loss: 0.3355 - val_accuracy: 0.9400 - val_loss: 0.1897


In [ ]:
# Input | Testing
image = tf.keras.preprocessing.image.load_img('dd.jpg', target_size=(128,128))
image_arr = tf.keras.preprocessing.image.img_to_array(image)
image_arr.shape

(128, 128, 3)

In [27]:
np_img_arr = np.expand_dims(image_arr, axis= 0)
np_img_arr.shape

(1, 128, 128, 3)

In [28]:

probability_prediction = model.predict(np_img_arr)
# For binary classification with sigmoid output, we apply a threshold
# If probability_prediction[0][0] >= 0.5, it's class 1 (dogs), otherwise class 0 (cats)
if probability_prediction[0][0] >= 0.5:
    prediction_class_index = 1  # Corresponds to 'dogs'
else:
    prediction_class_index = 0  # Corresponds to 'cats'

print(f"Raw probability: {probability_prediction[0][0]:.4f}")

class_names_map = {v: k for k, v in train_image.class_indices.items()}

# Use the index to look up the actual class name
predicted_class_name = class_names_map[prediction_class_index]

# Print the result
print(f"The predicted pet name is: {predicted_class_name}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Raw probability: 1.0000
The predicted pet name is: dogs
